In [1]:
from syft import nn
from syft import PhiTensor
from syft import GammaTensor
from syft import DataSubjectList
import numpy as np
from jax import numpy as jnp

/home/e/anaconda3/envs/Hagrid/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/e/PycharmProjects/PySyft/packages/syft/src/syft/core/common/serde/serializable.py:105: UserWarning: __attr_allowlist__ not defined for type Activation, even if it uses recursive serde, defaulting on the empty list.
  warnings.warn(
/home/e/PycharmProjects/PySyft/packages/syft/src/syft/core/common/serde/serializable.py:105: UserWarning: __attr_allowlist__ not defined for type Linear, even if it uses recursive serde, defaulting on the empty list.
  warnings.warn(
/home/e/PycharmProjects/PySyft/packages/syft/src/syft/core/common/serde/serializable.py:105: UserWarning: __attr_allowlist__ not defined for type BatchNorm, even if it uses recursive serde, defaulting on the empty list.
  warnings.warn(
/home/e/PycharmProjects/PySy

In [2]:
# third party
import numpy as np
from typing import Union

# relative
# from ....common.serde.serializable import serializable
# from ...autodp.phi_tensor import PhiTensor
XavierInitialization = nn.initializations.XavierInitialization
Layer = nn.layers.base.Layer

# @serializable(recursive_serde=True)
class Linear(Layer):
    def __init__(self, n_out, n_in=None):
        self.n_out = n_out
        self.n_in = n_in
        self.out_shape = (None, n_out)

        self.W = None
        self.b = None
        self.dW = None
        self.db = None
        self.last_input = None
        self.init = XavierInitialization()

    def connect_to(self, prev_layer=None):
        if prev_layer is None:
            assert self.n_in is not None
            n_in = self.n_in
        else:
            assert len(prev_layer.out_shape) == 2
            n_in = prev_layer.out_shape[-1]

        self.W = self.init((n_in, self.n_out))
        self.b = np.zeros((self.n_out,))

    def forward(self, input: Union[PhiTensor, GammaTensor], *args, **kwargs):
        self.last_input = input
        return input.dot(self.W) + self.b

    def backward(self, pre_grad: PhiTensor, *args, **kwargs):
        self.dW = self.last_input.transpose().dot(pre_grad)  # Should this transpose just the last 2 indices?
        #         self.dW = self.last_input.swapaxes(-1, -2)
        self.db = pre_grad.mean(axis=0)
        if not self.first_layer:
            return pre_grad.dot(self.W.T)

    @property
    def params(self):
        return self.W, self.b

    @property
    def grads(self):
        return self.dW, self.db


In [3]:
from syft import lazyrepeatarray as lra

sh = (10, 10)
dsl = DataSubjectList(
        one_hot_lookup=np.arange(10),
        data_subjects_indexed=np.random.choice(np.arange(10), size=sh)
    )
smol_data = GammaTensor(
        child=jnp.array(np.random.rand(*sh) * 255),
        data_subjects=dsl,
        min_vals=lra(0, shape=sh),
        max_vals=lra(255, shape=sh))

In [4]:
smol_data.dot(smol_data.transpose()).state.keys()

dict_keys(['1124648919', '1447917132'])

In [5]:
assert isinstance(smol_data.dot(np.ones((10,10))), GammaTensor)

In [6]:
lin = Linear(n_out=5)

In [7]:
lin.n_in = 10
lin.connect_to()

In [8]:
res = lin.forward(smol_data)

In [9]:
res.shape

(10, 5)

In [10]:
lin.W.shape

(10, 5)

In [11]:
res2 = lin.backward(res)

In [12]:
lin.W

array([[-0.08633093,  0.00612607, -0.2063987 , -0.33316428,  0.02381554],
       [ 0.00867286,  0.17949546,  0.46393389, -0.16510667, -0.60394843],
       [-0.38038538, -0.29532162, -0.55098174, -0.02299733,  0.03166108],
       [ 0.26568363,  0.36268732, -0.03565167, -0.49658959, -0.16534402],
       [-0.00979648,  0.20484431, -0.04689578, -0.55513685,  0.1242503 ],
       [-0.41465365, -0.17528505, -0.04197059,  0.24447768,  0.00236061],
       [ 0.33421524,  0.07858863,  0.17623702,  0.12178033, -0.38536195],
       [-0.59172267,  0.1283921 ,  0.20009202,  0.11012239, -0.23071266],
       [ 0.3875264 ,  0.26864924, -0.47968359,  0.50906667,  0.12847664],
       [-0.53181781, -0.20327994,  0.12539851, -0.51453117,  0.40068147]])

In [13]:
lin.b

array([0., 0., 0., 0., 0.])

In [14]:
a = np.ones((5,5))
b = a.mean(axis=1)

In [15]:
c = np.zeros_like(a)

In [16]:
d = c.reshape((*b.shape, -1))
d.shape

(5, 5)

In [17]:
b

array([1., 1., 1., 1., 1.])

In [18]:
b.shape

(5,)

In [19]:
b.reshape((1, *b.shape)).shape

(1, 5)

In [20]:
b_shape = np.expand_dims(b, 0).shape

In [21]:
d = c.reshape((-1, *b_shape))
d.shape

(5, 1, 5)

In [22]:
d

array([[[0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.]]])